In [1]:
# Importing necessary libraries
import re
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize
from collections import defaultdict
from collections import defaultdict
from gensim import corpora
from gensim.models import LdaModel, CoherenceModel
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import nltk
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import pprint
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
#Loading the dataset
df=pd.read_csv(r"all_fillings.csv")
#Displaying a few rows of the dataset
df.sample(5)

,ticker,filing_date,report_date,accession_no,text,has_esg_keyword
80,TSLA,2024-01-29,2023-12-31,0001628280-24-002390,...,True
361,ADBE,2019-01-25,2018-11-30,0000796343-19-000019,...,True
226,PEP,2018-02-13,2017-12-30,0000077476-18-000012,Table of Contents\n ...,True
248,WMT,2017-03-31,2017-01-31,0000104169-17-000021,...,True
118,JPM,2017-02-28,2016-12-31,0000019617-17-000314,...,True


In [3]:
df.shape

(543, 6)

In [4]:
#Printing the shape of the dataframe
print("Shape of the dataframe:", df.shape)
# Filtering out and keeping records where ESG mentions are True to presserve the scope of the analysis
print("Number of records with ESG mentions:", df[df['has_esg_keyword']==True].shape)

Shape of the dataframe: (543, 6)
Number of records with ESG mentions: (538, 6)


In [5]:
df.drop(columns=['has_esg_keyword'],inplace=True)

In [6]:
#Converting the filing_date column to datetime format
df['filing_date']=pd.to_datetime(df['filing_date'])

In [7]:
df=df[df['filing_date'].dt.year<=2024]


In [8]:
df.columns

Index(['ticker', 'filing_date', 'report_date', 'accession_no', 'text'], dtype='object')

In [9]:
df.drop(columns=['accession_no','report_date'],inplace=True)

In [10]:
# Declaring the scope of the analysis to be for last 5 years
print("Shape of the df before filtering by year:",df.shape)

Shape of the df before filtering by year: (497, 3)


In [11]:
#Checking for null values
df.isnull().sum()

ticker         0
filing_date    0
text           0
dtype: int64

In [12]:
df['ticker'].unique()

array(['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'TSLA', 'UNH',
       'XOM', 'JNJ', 'JPM', 'V', 'PG', 'MA', 'HD', 'CVX', 'MRK', 'ABBV',
       'KO', 'AVGO', 'PEP', 'COST', 'WMT', 'MCD', 'CSCO', 'TMO', 'ACN',
       'ABT', 'DHR', 'NKE', 'CRM', 'VZ', 'ADBE', 'CMCSA', 'WFC', 'TXN',
       'NEE', 'LIN', 'PM', 'NFLX', 'UPS', 'RTX', 'AMD', 'QCOM', 'ORCL',
       'HON', 'SPGI', 'LOW', 'INTU'], dtype=object)

In [14]:
# Printing a sample report text
pprint.pprint(df['text'].iloc[54][:1000])

('                                                                                                                   \n'
 '                                                   UNITED '
 'STATES                                                   \n'
 '                                                                                                                   \n'
 '                                                                                                                   \n'
 '                                        SECURITIES AND EXCHANGE '
 'COMMISSION                                         \n'
 '                                                                                                                   \n'
 '                                                                                                                   \n'
 '                                              Washington, D.C. '
 '20549                                               \n'
 '                  

In [15]:
# Cleaning the report_text column to remove boilerplate formatting
# Used Claude to generate the boilerplate patterns inorder to maintain consistency and to cover various boilerplate text
boilerplate=[r'UNITED STATES\s+SECURITIES AND EXCHANGE COMMISSION',
    r'Washington,?\s*D\.?C\.?\s*\d{5}',
    r'FORM 10-K',
    r'ANNUAL REPORT PURSUANT TO SECTION 13 OR 15\(d\)',
    r'SECURITIES EXCHANGE ACT OF 1934',
    r'For the fiscal year ended',
    r'Commission File Number:?\s*\d+-\d+',
    r'TRANSITION REPORT PURSUANT TO SECTION',
    r'I\.?R\.?S\.? Employer Identification No\.?',
    r'State or other jurisdiction of incorporation',
    r'Address of principal executive offices',
    r"Registrant's telephone number",
     r'☒|☐',
    r'\(Mark One\)',
    r'Yes ☒ No ☐',
    r'Indicate by check mark',
    r'TABLE OF CONTENTS',
    r'Item\s+\d+[A-Z]?\..*?\d+\s*$',  # Item entries with page numbers
    r'Page\s*$',
    r'Part\s+[IVX]+\s*$',
    r'Apple Inc\.\s*\|\s*\d{4}\s*Form\s*10-K\s*\|\s*\d+',
    r'Page\s+\d+\s+of\s+\d+',
    r'\n\s*\d+\s*\n',
    r'This Annual Report on Form 10-K.*?forward-looking statements?',
    r'contains forward-looking statements',
    r'Private Securities Litigation Reform Act of 1995',
    r'Forward-looking statements.*?historical or current fact',
    r'The Company assumes no obligation to revise or update',
    r'unless otherwise stated.*?refer to the Company.?s fiscal',
    r'See accompanying Notes to Consolidated Financial Statements\.?',
    r'incorporated by reference',
    r'DOCUMENTS INCORPORATED BY REFERENCE',
    r'filed with the U\.?S\.? Securities and Exchange Commission',
    r'available free of charge at investor\.apple\.com',
    r'SIGNATURES\s*$',
    r'Pursuant to the requirements of.*?Exchange Act',
    r'KNOW ALL PERSONS BY THESE PRESENTS',
    r'Power of Attorney',
    r'/s/.*?(?=\n)',  # Signature lines
    r'Principal Executive Officer',
    r'Principal Financial Officer',
    r'Principal Accounting Officer',
    r'EXHIBIT.*?(?=Item|$)',
    r'Exhibit Number.*?Filing Date',
    r'Incorporated by Reference',
    r'Exhibit Description',
    r'[╔╗╚╝═║╭╮╰╯─│┌┐└┘├┤┬┴┼]+',  # Box drawing characters
    r'─{3,}',  # Long dashes
    r'═{3,}',  # Double lines
    r'\.{3,}',
    r'Report of Independent Registered Public Accounting Firm',
    r'To the Shareholders and the Board of Directors',
    r'Opinion on.*?Financial Statements',
    r'Basis for Opinion',
    r'Critical Audit Matter',
    r'We have served as the Company.?s auditor since',
    r'\[Reserved\]',
    r'Not applicable\.?',
    r'None\.?\s*$',
    r'Index to Consolidated Financial Statements',
    r'Consolidated Statements? of',
    r'Notes to Consolidated Financial Statements',
    r'as of September \d+, \d{4}',
    ]

In [16]:
def clean_report_text(text):
    """
    Description:
    Cleans the report text by lowercasing the text, removing HTML tags and boilerplate patterns along with extra spaces and new lines.
    -----------
    Parameters:
    text (str): The report text to be cleaned.
    -----------
    Returns:
    cleaned_text(str): The cleaned report text.
    -----------
    Example Usage:
    cleaned_text=clean_report_text(text)
    -----------
    Raises:
    AssertionError: If the input text is not a string, is empty or is None.
    """
    #Input Validations
    assert isinstance(text,str),"Input text should be a string data type"
    assert len(text)>0,"Input text should not be empty."
    assert text is not None,"Input text should not be None."
    #Lowercasing the text
    cleaned_text=text.lower()
    # Removing 
    cleaned_text = re.sub(r'^[_]{3,}\s*$', '', cleaned_text, flags=re.MULTILINE)
    #Removing HTML tage
    cleaned_text=re.sub(r'<.*?>',' ',cleaned_text)
    #Removing boilerplate patterns
    for pattern in boilerplate:
        cleaned_text=re.sub(pattern,' ',cleaned_text)
    #Removing extra spaces and new lines
    cleaned_text=re.sub(r'\s+',' ',cleaned_text).strip()
    return cleaned_text

In [17]:
#Applying the clean_report_text function to the report_text column and storing the result in a new column named cleaned_report_text.
df['cleaned_report_text']=df['text'].apply(clean_report_text)

In [18]:
#Printing the cleaned_report_text
pprint.pprint(df['cleaned_report_text'].iloc[144][:1000])

('united states securities and exchange commission washington, d.c. 20549 form '
 '10-k annual report pursuant to section 13 or 15(d) of the securities '
 'exchange act of 1934 for the fiscal year ended december 31, 2021 or '
 'transition report pursuant to section 13 or 15(d) of the securities exchange '
 'act of 1934 for the transition period from to commission file number: '
 '001-32877 mastercard incorporated (exact name of registrant as specified in '
 'its charter) delaware 13-4172551 (state or other jurisdiction of -irs '
 'employer incorporation or organization) identification number 2000 purchase '
 'street purchase, ny 10577 (address of principal executive offices) -zip code '
 '( 914) 249-2000 (registrant’s telephone number, including area code) '
 'securities registered pursuant to section 12(b) of the act: title of each '
 'class trading symbol name of each exchange of which registered class a '
 'common stock, par value $0.0001 ma new york stock exchange per share 1.1% '


In [19]:
df['cleaned_report_text'].nunique()

497

In [20]:
#### Filtering out ESG sentences from the cleaned_report_text column using a dictionary of ESG-related keywords.
#### (GEN AI)Comprehensive ESG Keyword Dictionary from S&P 500 Companies' Reports
esg_keywords = {
    'E': {  # Environmental
        'emissions': ['greenhouse gas', 'ghg emissions', 'carbon emissions', 'co2 emissions',
                     'scope 1', 'scope 2', 'scope 3', 'emissions reduction', 'carbon footprint',
                     'emissions intensity', 'methane emissions', 'fugitive emissions'],
        'energy': ['renewable energy', 'clean energy', 'energy efficiency', 'energy consumption',
                  'solar power', 'wind energy', 'energy intensity', 'fossil fuel', 'coal',
                  'natural gas', 'electricity consumption', 'energy management'],
        'climate': ['climate change', 'climate risk', 'climate action', 'climate strategy',
                   'net zero', 'carbon neutral', 'decarbonization', 'global warming',
                   'climate resilience', 'climate goals', 'climate target'],
        'water': ['water consumption', 'water management', 'water efficiency', 'water withdrawal',
                 'water recycling', 'water stress', 'wastewater', 'water conservation',
                 'water intensity', 'freshwater'],
        'waste': ['waste management', 'waste reduction', 'waste recycling', 'hazardous waste',
                 'landfill', 'waste diversion', 'circular economy', 'zero waste',
                 'waste generated', 'waste intensity'],
        'biodiversity': ['biodiversity', 'ecosystem', 'deforestation', 'habitat protection',
                        'endangered species', 'nature conservation', 'land use'],
        'pollution': ['air pollution', 'water pollution', 'soil contamination', 'plastic waste',
                     'toxic substances', 'pollutants', 'environmental impact']
    },

    'S': {  # Social
        'labor': ['employee', 'workforce', 'labor practices', 'working conditions', 'fair wages',
                 'living wage', 'compensation', 'benefits', 'collective bargaining', 'union',
                 'employee turnover', 'retention rate', 'employee satisfaction'],
        'diversity': ['diversity', 'inclusion', 'gender diversity', 'racial diversity',
                     'ethnic diversity', 'women in leadership', 'minority representation',
                     'equal opportunity', 'pay equity', 'wage gap', 'diverse workforce'],
        'health_safety': ['occupational health', 'workplace safety', 'injury rate', 'fatality',
                         'lost time injury', 'safety training', 'health and safety',
                         'accident rate', 'safety performance', 'worker safety'],
        'training': ['employee training', 'professional development', 'skill development',
                    'training hours', 'training programs', 'education programs', 'career development',
                    'upskilling', 'reskilling'],
        'human_rights': ['human rights', 'labor rights', 'child labor', 'forced labor',
                        'modern slavery', 'human trafficking', 'fair treatment',
                        'workers rights'],
        'community': ['community engagement', 'social impact', 'community investment',
                     'philanthropy', 'charitable giving', 'local communities',
                     'community development', 'social responsibility'],
        'customer': ['customer satisfaction', 'product safety', 'data privacy',
                    'customer protection', 'responsible marketing', 'product quality',
                    'consumer rights'],
        'supply_chain': ['supply chain', 'supplier diversity', 'responsible sourcing',
                        'supplier assessment', 'supply chain sustainability']
    },

    'G': {  # Governance
        'board': ['board of directors', 'board diversity', 'board independence',
                 'board composition', 'board oversight', 'board committees',
                 'director', 'independent director', 'board meeting'],
        'ethics': ['code of conduct', 'ethics', 'integrity', 'anti-corruption',
                  'anti-bribery', 'whistleblower', 'compliance', 'ethical behavior',
                  'business ethics'],
        'risk': ['risk management', 'risk assessment', 'enterprise risk',
                'risk oversight', 'risk mitigation', 'internal controls',
                'risk framework'],
        'transparency': ['transparency', 'disclosure', 'reporting', 'accountability',
                        'stakeholder engagement', 'materiality assessment',
                        'esg reporting', 'sustainability report'],
        'executive_comp': ['executive compensation', 'ceo pay', 'pay ratio',
                          'incentive compensation', 'performance-based pay',
                          'compensation committee'],
        'shareholder': ['shareholder rights', 'shareholder engagement', 'voting rights',
                       'shareholder proposal', 'proxy voting', 'shareholder value'],
        'cybersecurity': ['cybersecurity', 'data security', 'information security',
                         'cyber risk', 'data breach', 'security measures',
                         'data protection'],
        'regulatory': ['regulatory compliance', 'legal compliance', 'regulatory risk',
                      'compliance program', 'audit', 'internal audit']
    }
}

In [21]:
# Storing all ESG keywords in a single list with categories
all_esg_keywords=[]
dim_keywords={}
for category,subcat in esg_keywords.items():
    for subcat, terms in esg_keywords[category].items():
        all_esg_keywords.extend(terms)
        for term in terms:
            dim_keywords[term] = (category, subcat)


In [22]:
# Tokenizing the cleaned_report_text into sentences and storing them in a new column named cleaned_sentences.
df['cleaned_sentences']=df['cleaned_report_text'].apply(sent_tokenize)

In [23]:
# Creating a function to filter ESG sentences from the tokenized sentences and keeping surrounding sentences for context
def filtering_esg_sentences(sentences, all_esg_keywords, context_window=1):
    """
    Description:
    Filters ESG-related sentences from a list of sentences and includes contextual sentences based on the specified context window.
    -----------
    Parameters:
    sentences (list): List of sentences to filter.
    all_esg_keywords (list): List of ESG-related keywords to search for in the sentences.
    context_window (int): Number of surrounding sentences to include for context.
    -----------
    Returns:
    list: List of ESG-related sentences with context.
    -----------
    Example Usage:
    esg_sentences=filtering_esg_sentences(sentences,context_window=2)
    -----------
    Raises:
    AssertionError: If the input sentences is not a list or if any element in sentences is not a string.
    AssertionError: If the input all_esg_keywords is not a list or if any element in all_esg_keywords is not a string.
    """
    assert isinstance(sentences,list),"Input sentences should be a list data type."
    assert all(isinstance(sent,str) for sent in sentences),"All elements in sentences should be strings."
    assert isinstance(all_esg_keywords,list),"Input all_esg_keywords should be a list data type."
    assert all(isinstance(k,str) for k in all_esg_keywords),"All elements in all_esg_keywords should be strings."
    esg_sentences=[]
    for i,sent in enumerate(sentences):
        if any(k in sent for k in all_esg_keywords):
            start=max(0,i-context_window)
            end=min(len(sentences),i+context_window+1)
            esg_sentences.extend(sentences[start:end])
    #Removing duplicates and returning as a single string
    return list(set(esg_sentences))

In [24]:
#Applying the filtering_esg_sentences function to the cleaned_sentences column to extract ESG-related sentences
df['ESG_sentences']=df['cleaned_sentences'].apply(lambda x: filtering_esg_sentences(x, all_esg_keywords=all_esg_keywords))

In [25]:
# Creating regrex patterns for each ESG dimension for faster matching
def dim_matching(keywords):
    """
    Creates a regex pattern for matching ESG keywords.
    -----------
    Parameters:
    keywords (list): List of ESG-related keywords.
    -----------
    Returns:
    re.Pattern: Compiled regex pattern for matching ESG keywords.
    -----------
    Example Usage:
    pattern=dim_matching(keywords)
    -----------
    Raises:
    AssertionError: If the input keywords is not a list or if any element in keywords is not a string.
    """
    assert isinstance(keywords,list),"Input keywords should be a list data type."
    assert all(isinstance(k,str) for k in keywords),"All elements in keywords should be strings."
    dim=[r'\b'+re.escape(keyw)+r'\b'for keyw in keywords]
    matching=re.compile('|'.join(dim),re.IGNORECASE)
    return matching


In [26]:
#Mappig each ESG dimension to its corresponding regex pattern
dim_pattern_mapping={}
for dim,dim_words in esg_keywords.items():
    for sub_cat,words in dim_words.items():
        dim_pattern_mapping[(dim,sub_cat)]=dim_matching(words)
print(dim_pattern_mapping)

{('E', 'emissions'): re.compile('\\bgreenhouse\\ gas\\b|\\bghg\\ emissions\\b|\\bcarbon\\ emissions\\b|\\bco2\\ emissions\\b|\\bscope\\ 1\\b|\\bscope\\ 2\\b|\\bscope\\ 3\\b|\\bemissions\\ reduction\\b|\\bcarbon\\ footprint\\b|\\bemi, re.IGNORECASE), ('E', 'energy'): re.compile('\\brenewable\\ energy\\b|\\bclean\\ energy\\b|\\benergy\\ efficiency\\b|\\benergy\\ consumption\\b|\\bsolar\\ power\\b|\\bwind\\ energy\\b|\\benergy\\ intensity\\b|\\bfossil\\ fuel\\b|\\bcoal\\b|\\bn, re.IGNORECASE), ('E', 'climate'): re.compile('\\bclimate\\ change\\b|\\bclimate\\ risk\\b|\\bclimate\\ action\\b|\\bclimate\\ strategy\\b|\\bnet\\ zero\\b|\\bcarbon\\ neutral\\b|\\bdecarbonization\\b|\\bglobal\\ warming\\b|\\bclimate\\ resilienc, re.IGNORECASE), ('E', 'water'): re.compile('\\bwater\\ consumption\\b|\\bwater\\ management\\b|\\bwater\\ efficiency\\b|\\bwater\\ withdrawal\\b|\\bwater\\ recycling\\b|\\bwater\\ stress\\b|\\bwastewater\\b|\\bwater\\ conservation\\b|\\bwater\, re.IGNORECASE), ('E', 'waste

In [27]:
# Adding a column ESG dimension mapping each ESG sentence to its corresponding dimension and sub-category
def map_esg_dimensions(esg_sentences):
    """
    Maps ESG sentences with their corresponding ESG dimensions and sub-categories.
    -----------
    Parameters:
    esg_sentences (list): List of ESG-related sentences.
    -----------
    Returns:
    list: List of tuples containing ESG sentence, dimension, and sub-category.
    -----------
    Example Usage:
    esg_dim_mapping=map_esg_dimensions(esg_sentences)
    -----------
    Raises:
    AssertionError: If the input esg_sentences is not a list or if any element in esg_sentences is not a string.
    """
    matched_keywords=[]
    assert isinstance(esg_sentences,list),"Input esg_sentences should be a list data type."
    assert all(isinstance(sent,str) for sent in esg_sentences),"All elements in esg_sentences should be strings."
    for (dim,subcat),pattern in dim_pattern_mapping.items():
        if any(pattern.search(sent)for sent in esg_sentences):
            matched_keywords.append((dim,subcat))
    return matched_keywords

In [28]:
# ESG dimension mapping for each report in order to analyze which dimensions are most frequently discussed
df['ESG_dimension']=df['ESG_sentences'].apply(map_esg_dimensions)

In [30]:
# Random sample of ESG dimensions mapped for a report
df[['ticker','filing_date','ESG_dimension']].sample(5)

,ticker,filing_date,ESG_dimension
326,NKE,2017-07-20,"[(E, climate), (S, labor), (S, diversity), (S,..."
218,AVGO,2020-12-18,"[(E, energy), (E, climate), (E, pollution), (S..."
379,WFC,2015-02-25,"[(S, labor), (G, board), (G, ethics), (G, risk..."
219,AVGO,2021-12-17,"[(E, energy), (E, climate), (E, pollution), (S..."
19,MSFT,2024-07-30,"[(E, emissions), (E, energy), (E, climate), (E..."


In [82]:
df['ESG_dimension'].iloc[144]

[('E', 'climate'),
 ('E', 'biodiversity'),
 ('S', 'labor'),
 ('S', 'diversity'),
 ('S', 'health_safety'),
 ('S', 'training'),
 ('S', 'human_rights'),
 ('S', 'community'),
 ('S', 'supply_chain'),
 ('G', 'board'),
 ('G', 'ethics'),
 ('G', 'risk'),
 ('G', 'transparency'),
 ('G', 'executive_comp'),
 ('G', 'shareholder'),
 ('G', 'cybersecurity'),
 ('G', 'regulatory')]

In [32]:
pprint.pprint(df['ESG_sentences'].iloc[144][:100])

['see note 21 (legal and regulatory proceedings) for additional information '
 'regarding the company’s accrued litigation.',
 'in addition, issuers could seek a fee reduction from us to decrease the '
 'expense of their payment programs, particularly if regulation has a '
 'disproportionate impact on us as compared to our competitors in terms of the '
 'fees we can charge.',
 'we provide more detailed information regarding our employees, including '
 'additional workforce demographics such as gender and racial/ethnic '
 'representation, in our sustainability report, our proxy statement, our '
 'global inclusion report and our u.s. consolidated eeo-1 report, all of which '
 'are located on our website.',
 'the components of deferred tax assets and liabilities at december 31 are as '
 'follows: deferred tax assets (in millions) 2020 accrued liabilities $ 497 $ '
 '324 compensation and benefits 260 218 state taxes and other credits 40 47 '
 'net operating and capital losses 136 147 unrea

In [33]:
# Based on the ESG dimensions, finding the % distribution of Environmental, Social and Governance mentions across all reports.
esg_list=[]
#Calculating the percentage distribution of ESG dimensions for each report
for index,row in df.iterrows():
    total_sentences=len(row['ESG_dimension'])
    if total_sentences==0:
        e_percentage=s_percentage=g_percentage=0
    else:
        e_count=sum(1 for dim,subcat in row['ESG_dimension'] if dim=='E')
        s_count=sum(1 for dim,subcat in row['ESG_dimension'] if dim=='S')
        g_count=sum(1 for dim,subcat in row['ESG_dimension'] if dim=='G')
        e_percentage=(e_count/total_sentences)*100
        s_percentage=(s_count/total_sentences)*100
        g_percentage=(g_count/total_sentences)*100
    esg_list.append({'ticker':row['ticker'],
                     'filing_date':row['filing_date'],
                     'ESG_sentences':row['ESG_sentences'],
                     'ESG_dimension':row['ESG_dimension'],
                     'E_percentage':round(e_percentage,2),
                     'S_percentage':round(s_percentage,2),
                     'G_percentage':round(g_percentage,2)
    })
esg_percentage_dist=pd.DataFrame(esg_list)

In [34]:
esg_percentage_dist.head()

,ticker,filing_date,ESG_sentences,ESG_dimension,E_percentage,S_percentage,G_percentage
0,AAPL,2015-10-28,[further information regarding share-based com...,"[(E, climate), (E, biodiversity), (S, labor), ...",15.38,30.77,53.85
1,AAPL,2016-10-26,[item 12. security ownership of certain benefi...,"[(E, climate), (E, biodiversity), (S, labor), ...",15.38,30.77,53.85
2,AAPL,2017-11-03,"[officer’s certificate of the registrant, date...","[(E, climate), (E, biodiversity), (S, labor), ...",15.38,30.77,53.85
3,AAPL,2018-11-05,[while the company maintains directors and off...,"[(E, climate), (E, biodiversity), (S, labor), ...",14.29,35.71,50.00
4,AAPL,2019-10-31,[item 11. executive compensation the informati...,"[(E, climate), (E, biodiversity), (S, labor), ...",15.38,30.77,53.85


In [35]:
def prepare_seed_topics_gensim(dimension):
    """
    Creating seed topics from ESG dictionary for gensim. It is enhanced for E dimension to catch more environmental content as it was found to be less represented in initial topic modeling runs.
    -----------
    Parameters:
    dimension (str): ESG dimension ('E', 'S', or 'G').
    -----------
    Returns:
    dict: Dictionary of seed topics for the specified ESG dimension.
    -----------
    Example Usage:
    seed_topics=prepare_seed_topics_gensim('E')
    -----------
    Raises:
    AssertionError: If the input dimension is not one of 'E', 'S', or 'G'.
    """
    assert dimension in ['E','S','G'],"Input dimension should be one of 'E', 'S', or 'G'."
    # Focusing on E dimension with more granular seed topics
    if dimension == 'E':
        seed_topics = {
            'Climate & Emissions': [
                # Core climate terms
                'emissions', 'carbon', 'greenhouse', 'ghg', 'co2', 'scope',
                'emission', 'methane', 'decarbonization', 'footprint', 'climate',
                'warming', 'carbon dioxide', 'gases', 'intensity',
                # Action & strategy terms
                'reduction', 'targets', 'goals', 'net zero', 'neutrality',
                'mitigation', 'offset', 'sequestration', 'capture',
                # Measurement & reporting
                'inventory', 'reporting', 'disclosure', 'measurement', 'baseline',
                'tracking', 'monitoring', 'verification', 'science based'
            ],
            'Energy & Resources': [
                # Energy sources
                'renewable', 'energy', 'solar', 'wind', 'clean', 'fossil',
                'electricity', 'power', 'coal', 'natural gas', 'fuel',
                # Energy transition
                'transition', 'electrification', 'efficiency', 'consumption',
                'generation', 'grid', 'storage', 'battery', 'hydrogen',
                # Resources
                'water', 'waste', 'recycling', 'circular', 'conservation',
                'wastewater', 'resource', 'materials', 'raw materials'
            ],
            'Environmental Management': [
                # Waste & circular economy
                'waste', 'recycling', 'circular economy', 'landfill', 'diversion',
                'reuse', 'reduce', 'compost', 'hazardous waste',
                # Biodiversity & ecosystems
                'biodiversity', 'ecosystem', 'habitat', 'species', 'deforestation',
                'land use', 'nature', 'conservation', 'restoration',
                # Pollution & impact
                'pollution', 'contamination', 'toxic', 'spills', 'discharge',
                'air quality', 'environmental impact', 'sustainability'
            ]
        }
    # Focusing on S dimension with key social topics
    elif dimension == 'S':
        seed_topics = {
            'Workforce & Labor': [
                'employee', 'workforce', 'labor', 'workers', 'employment',
                'hiring', 'talent', 'retention', 'compensation', 'wage',
                'turnover', 'benefits', 'collective bargaining', 'union'
            ],
            'Diversity & Inclusion': [
                'diversity', 'inclusion', 'equity', 'gender', 'representation',
                'minority', 'underrepresented', 'equal', 'discrimination',
                'women', 'race', 'ethnicity', 'pay equity', 'belonging'
            ],
            'Health & Safety': [
                'safety', 'health', 'workplace', 'injury', 'wellbeing',
                'training', 'accident', 'hazard', 'protection', 'occupational',
                'fatality', 'incident', 'wellness', 'mental health'
            ]
        }
    # Focusing on G dimension with key governance topics
    elif dimension == 'G':
        seed_topics = {
            'Board & Leadership': [
                'board', 'directors', 'governance', 'leadership', 'oversight',
                'committee', 'executive', 'independence', 'chairman',
                'compensation committee', 'audit committee', 'nomination'
            ],
            'Ethics & Compliance': [
                'ethics', 'compliance', 'integrity', 'transparency', 'corruption',
                'bribery', 'fraud', 'whistleblower', 'code of conduct',
                'anti corruption', 'conflict of interest', 'accountability'
            ],
            'Risk & Security': [
                'risk', 'cybersecurity', 'data', 'privacy', 'security',
                'breach', 'vulnerability', 'threat', 'audit', 'control',
                'risk management', 'data protection', 'gdpr', 'compliance'
            ]
        }
    return seed_topics

In [36]:
def preprocess_for_gensim(df, dimension):
    """
    Prepares documents and metadata for Gensim topic modeling based on the specified ESG dimension.
    -----------
    Parameters:
    dataframe (pd.DataFrame): DataFrame containing ESG sentences and dimensions.
    dimension (str): ESG dimension ('E', 'S', or 'G').
    -----------
    Returns:
    tuple: Tuple containing list of documents and their metadata.
    -----------
    Example Usage:
    documents, metadata = preprocess_for_gensim(dataframe, 'E')
    -----------
    Raises:
    AssertionError: If the input dataframe is not a pandas DataFrame.
    """
    assert isinstance(df, pd.DataFrame), "Input dataframe should be a pandas DataFrame."
    # Preparing empty lists to store documents and metadata
    documents = []
    metadata = []
    # Iterating through each report in the dataframe
    for idx, row in df.iterrows():
        report_text = []
        # Collect sentences for this dimension and sub-categories
        for i, (dim, subcat) in enumerate(row['ESG_dimension']):
            if dim == dimension and i < len(row['ESG_sentences']):
                report_text.append(row['ESG_sentences'][i])
        if dimension == 'E' and len(report_text) < 5:
            # Environmental keywords to look for
            env_keywords = ['emission', 'carbon', 'climate', 'energy', 'renewable', 
                          'waste', 'water', 'pollution', 'environmental', 'sustainability',
                          'ghg', 'greenhouse', 'footprint', 'recycling', 'biodiversity']
            # Check all sentences for environmental content
            for i, sentence in enumerate(row['ESG_sentences']):
                sentence_lower = sentence.lower()
                if any(keyword in sentence_lower for keyword in env_keywords):
                    if sentence not in report_text:
                        report_text.append(sentence)
        if report_text:
            doc_text = ' '.join(report_text)
            documents.append(doc_text)
            metadata.append({
                'ticker': row['ticker'],
                'filing_date': row['filing_date'],
                'sentence_count': len(report_text)
            })
    return documents, metadata

In [37]:
def tokenize_documents(documents, dimension='E'):
    """
    Tokenizes and preprocesses documents for Gensim topic modeling, with enhanced stopword removal for ESG context.
    -----------
    Parameters:
    documents (list): List of document strings to be tokenized.
    dimension (str): ESG dimension ('E', 'S', or 'G') for customized stopword removal.
    -----------
    Returns:
    list: List of tokenized and preprocessed documents.
    -----------
    Example Usage:
    tokenized_docs = tokenize_documents(documents, 'E')
    -----------
    Raises:
    AssertionError: If the input documents is not a list or if any element in documents is not a string.
    """
    assert isinstance(documents, list), "Input documents should be a list data type."
    assert all(isinstance(doc, str) for doc in documents), "All elements in documents should be strings."
    # Adding custom stopwords relevant to ESG context which were found to be non-informative during initial analysis
    custom_stopwords = STOPWORDS.union({
        # Generic business terms
        'company', 'business', 'year', 'fiscal', 'including', 
        'related', 'may', 'also', 'could', 'would', 'new',
        'operations', 'activities', 'process', 'processes',
        'products', 'services', 'provide', 'support',
        # Time references
        'period', 'quarter', 'annual', 'date', 'time',
        # Generic verbs
        'make', 'use', 'used', 'using', 'continue', 'increase',
        'develop', 'improve', 'ensure', 'maintain', 'implement',
        # Filler words
        'various', 'multiple', 'number', 'level', 'levels',
        'certain', 'additional', 'significant', 'range',
        # Generic nouns that don't add meaning
        'efforts', 'practices', 'measures', 'initiatives',
        'programs', 'system', 'systems', 'approach'
    })
    # For E dimension, keeping these important terms even if they might seem generic
    if dimension == 'E':
        keep_terms = {'emissions', 'emission', 'carbon', 'energy', 'waste', 
                     'water', 'climate', 'environmental', 'renewable'}
        custom_stopwords = custom_stopwords - keep_terms
    processed_docs = []
    for doc in documents:
        # Simple preprocessing: lowercase, remove punctuation, tokenize
        tokens = simple_preprocess(doc, deacc=True)
        # Remove stopwords and short tokens
        tokens = [token for token in tokens if token not in custom_stopwords and len(token) > 3]
        processed_docs.append(tokens)
    return processed_docs

In [38]:
def create_eta_prior_for_seeding(dictionary, seed_topics, num_topics, seed_weight=0.5):
    """
    Creating an eta prior matrix for Gensim LDA topic modeling to seed topics with specified seed words.
    -----------
    Parameters:
    dictionary (gensim.corpora.Dictionary): Gensim dictionary object.
    seed_topics (dict): Dictionary of seed topics with topic names as keys and lists of seed words as values.
    num_topics (int): Total number of topics in the LDA model.
    seed_weight (float): Weight to assign to seed words in the eta prior matrix.
    -----------
    Returns:
    np.ndarray: Eta prior matrix for Gensim LDA topic modeling.
    -----------
    Example Usage:
    eta = create_eta_prior_for_seeding(dictionary, seed_topics, num_topics, seed_weight=0.5)
    -----------
    Raises:
    AssertionError: If the input dictionary is not a gensim Dictionary object.
    AssertionError: If the input seed_topics is not a dictionary.
    """
    assert isinstance(dictionary, corpora.Dictionary), "Input dictionary should be a gensim Dictionary object."
    assert isinstance(seed_topics, dict), "Input seed_topics should be a dictionary."
    vocab_size = len(dictionary)
    # Initializing eta matrix with a symmetric prior
    eta = np.full((num_topics, vocab_size), 0.01)
    # Boosting seed words for their respective topics
    for topic_idx, (topic_name, seed_words) in enumerate(seed_topics.items()):
        if topic_idx >= num_topics:
            break
        for word in seed_words:
            if word in dictionary.token2id:
                word_id = dictionary.token2id[word]
                # Assign higher weight to seed words
                eta[topic_idx, word_id] = seed_weight  
    return eta

In [39]:
def guided_lda_gensim(dataframe, dimension, esg_keywords, 
                     num_topics=3, seed_weight=0.5, passes=50, iterations=400,
                     min_word_freq=1): 
    """
    Performs Guided LDA topic modeling using Gensim with seed topics based on ESG keywords.
    -----------
    Parameters:
    dataframe (pd.DataFrame): DataFrame containing ESG sentences and dimensions.
    dimension (str): ESG dimension ('E', 'S', or 'G').
    esg_keywords (dict): Dictionary of ESG-related keywords.
    num_topics (int): Number of topics for LDA model.
    seed_weight (float): Weight to assign to seed words in the eta prior matrix.
    passes (int): Number of passes through the corpus during training.
    iterations (int): Number of iterations for each pass.
    min_word_freq (int): Minimum frequency for words to be included in the dictionary.
    Returns:
    tuple: Tuple containing the trained LDA model, coherence score, topics DataFrame, dictionary, corpus, and processed documents.
    -----------
    Example Usage:
    lda_model, coherence_score, topics_df, dictionary, corpus, processed_docs = guided_lda_gensim(
        dataframe, 'E', esg_keywords, num_topics=3, seed_weight=0.5, passes=50, iterations=400
    )
    -----------
    Raises:
    AssertionError: If the input dataframe is not a pandas DataFrame.
    AssertionError: If the input dimension is not one of 'E', 'S', or
    """
    assert isinstance(dataframe, pd.DataFrame), "Input dataframe should be a pandas DataFrame."
    assert dimension in ['E', 'S', 'G'], "Input dimension should be one of 'E', 'S', or 'G'."
    # 1. Preparing documents by filtering based on ESG dimension
    documents, metadata = preprocess_for_gensim(dataframe, dimension)
    if len(documents) == 0:
        print(f" No documents found for {dimension}")
        return None, None, None, None, None, None
    # 2. Tokenizing documents by removing stopwords and irrelevant terms
    processed_docs = tokenize_documents(documents, dimension)
    # 3. Creating dictionary and corpus which are consisting of bag-of-words representations of the documents 
    dictionary = corpora.Dictionary(processed_docs)
    # Special handling for E dimension to retain more words as it has less content
    if dimension == 'E':
        dictionary.filter_extremes(no_below=min_word_freq, no_above=0.95, keep_n=1500)
    else:
        dictionary.filter_extremes(no_below=2, no_above=0.9, keep_n=1000)
    corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
    # 4. Preparing seed topics for the specified ESG dimension which will guide the LDA model to focus on relevant themes
    seed_topics = prepare_seed_topics_gensim(dimension)
    # 5. Creating eta prior matrix for seeding topics which influences the topic-word distributions during model training and helps in steering the topics towards desired themes
    eta = create_eta_prior_for_seeding(dictionary, seed_topics, num_topics, seed_weight)
    # 6. Training Guided LDA model with the eta prior to incorporate seed topics
    lda_model = LdaModel(
        corpus=corpus,
        id2word=dictionary,
        num_topics=num_topics,
        random_state=42,
        passes=passes,
        iterations=iterations,
        eta=eta, 
        alpha='auto',
        per_word_topics=True,
        minimum_probability=0.0
    )
    print(f"LDA model trained for {dimension} dimension with {num_topics} topics.")
    # 7. Calculating coherence score which measures the semantic similarity of words within topics to evaluate the quality of the topics generated by the LDA model
    coherence_model = CoherenceModel(
        model=lda_model,
        texts=processed_docs,
        dictionary=dictionary,
        coherence='c_v'
    )
    coherence_score = coherence_model.get_coherence()
    print(f"Coherence score: {coherence_score:.4f}")
    # 8. Extracting and displaying topics with seed vs discovered words
    topics = []
    topic_names = list(seed_topics.keys())
    print(f"DISCOVERED TOPICS - {dimension}")    
    for topic_id in range(num_topics):
        topic_name = topic_names[topic_id] if topic_id < len(topic_names) else f"Topic {topic_id}"
        # Get top words for the topic
        top_words = lda_model.show_topic(topic_id, topn=20)
        words = [word for word, prob in top_words]
        probs = [prob for word, prob in top_words]
        # Identify seed vs discovered words
        seed_words_for_topic = seed_topics[topic_name] if topic_id < len(seed_topics) else []
        seed_words_in_top = [w for w in words if w in seed_words_for_topic]
        discovered_words = [w for w in words if w not in seed_words_for_topic]
        # Display topic details
        print(f" {topic_name}")
        print(f"Seed words ({len(seed_words_in_top)}): {', '.join(seed_words_in_top)}")
        print(f"Discovered ({len(discovered_words)}): {', '.join(discovered_words[:12])}")
        print(f"Top 5 by weight: {', '.join([f'{w}({p:.3f})' for w, p in zip(words[:5], probs[:5])])}")
        # Storing topic details in a list for DataFrame creation
        topics.append({
            'Dimension': dimension,
            'Topic_ID': f'{dimension}_T{topic_id}',
            'Topic_Name': topic_name,
            'Top_Words': ', '.join(words[:15]),
            'Seed_Words_In_Top': ', '.join(seed_words_in_top),
            'Discovered_Words': ', '.join(discovered_words[:10]),
            'Coherence': coherence_score
        })
    topics_df = pd.DataFrame(topics)
    # 9. Getting document-topic distributions
    doc_topics_list = []
    # Preparing document-topic distributions for each document in the corpus
    for doc_id, doc_bow in enumerate(corpus):
        topic_dist = lda_model.get_document_topics(doc_bow, minimum_probability=0.0)
        topic_probs = [prob for _, prob in sorted(topic_dist, key=lambda x: x[0])]
        doc_topics_list.append({
            'ticker': metadata[doc_id]['ticker'],
            'filing_date': metadata[doc_id]['filing_date'],
            **{f'{dimension}_T{i}': topic_probs[i] for i in range(num_topics)},
            'dominant_topic': np.argmax(topic_probs),
            'dominant_topic_name': topic_names[np.argmax(topic_probs)] if np.argmax(topic_probs) < len(topic_names) else f'Topic {np.argmax(topic_probs)}',
        })
    doc_topics_df = pd.DataFrame(doc_topics_list)
    return lda_model, corpus, dictionary, topics_df, doc_topics_df, metadata


In [40]:
def create_pyldavis_gensim(lda_model, corpus, dictionary, dimension):
    """
    Create interactive pyLDAvis visualization for Gensim LDA model.
    -----------
    Parameters:
    lda_model (gensim.models.LdaModel): Trained Gensim LDA model.
    corpus (list): Gensim corpus used for training the LDA model.
    dictionary (gensim.corpora.Dictionary): Gensim dictionary used for training the LDA model.
    dimension (str): ESG dimension ('E', 'S', or 'G').
    -----------
    Returns:
    pyLDAvis prepared visualization object.
    -----------
    Example Usage:
    vis = create_pyldavis_gensim(lda_model, corpus, dictionary, 'E')
    """
    # Preparing pyLDAvis visualization for the LDA model using t-SNE for dimensionality reduction
    vis = gensimvis.prepare(
        lda_model,
        corpus,
        dictionary,
        mds='tsne',
        sort_topics=False
    )
    return vis

In [41]:
# Storing all results for each ESG dimension
all_results = {}
# Running guided LDA topic modeling for each ESG dimension with customized parameters
for dim in ['E', 'S', 'G']:
    # For E dimension, using more lenient parameters to capture more content
    if dim == 'E':
        # More lenient for E dimension
        lda_model, corpus, dictionary, topics_df, doc_topics_df, metadata = guided_lda_gensim(
            df,
            dimension=dim,
            esg_keywords=esg_keywords,
            num_topics=3,
            seed_weight=0.4,  # Moderate guidance
            passes=60,  # More passes for E
            iterations=500,
            min_word_freq=1  # Keep rare words
        )
    else:
        lda_model, corpus, dictionary, topics_df, doc_topics_df, metadata = guided_lda_gensim(
            df,
            dimension=dim,
            esg_keywords=esg_keywords,
            num_topics=3,
            seed_weight=0.3,
            passes=50,
            iterations=400,
            min_word_freq=2
        )
    # When the model is successfully created, storing the results and generating visualizations
    if lda_model:
        # Store results
        all_results[dim] = {
            'model': lda_model,
            'corpus': corpus,
            'dictionary': dictionary,
            'topics': topics_df,
            'doc_topics': doc_topics_df,
            'metadata': metadata
        }
        vis = create_pyldavis_gensim(lda_model, corpus, dictionary, dim)
        filename = f'LDA_Visualization_{dim}.html'
        pyLDAvis.save_html(vis, filename)
        print(f"Saved: {filename}")
#Executing the topic modeling for each ESG dimension and saving the visualizations
for dim in ['E', 'S', 'G']:
    if dim in all_results:
        print(f"LDA_Visualization_{dim}.html - Interactive topic visualization")
        topics_df = all_results[dim]['topics']
        for _, row in topics_df.iterrows():
            print(f"{row['Dimension']} Topic {row['Topic_ID'][-1]}: {row['Top_Words']}")

LDA model trained for E dimension with 3 topics.
Coherence score: 0.4339
DISCOVERED TOPICS - E
 Climate & Emissions
Seed words (1): climate
Discovered (19): environmental, laws, regulations, financial, costs, product, changes, health, change, compliance, safety, subject
Top 5 by weight: environmental(0.026), laws(0.015), regulations(0.014), financial(0.010), costs(0.010)
 Energy & Resources
Seed words (3): energy, solar, storage
Discovered (17): december, total, million, notes, assets, financial, income, cash, liabilities, share, statements, costs
Top 5 by weight: energy(0.028), december(0.016), total(0.015), million(0.012), notes(0.011)
 Environmental Management
Seed words (1): sustainability
Discovered (19): financial, energy, climate, stock, changes, change, information, global, president, natural, firm, equity
Top 5 by weight: financial(0.013), energy(0.013), climate(0.012), stock(0.008), changes(0.007)
Saved: LDA_Visualization_E.html
LDA model trained for S dimension with 3 topics

Governance-related topics exhibit the highest semantic clarity (coherence = 0.4918), followed by Social (0.4402) and Environmental (0.4339). This suggests that corporate disclosures are most precise and structured when discussing governance aspects, while environmental reporting remains more diffuse and contextually blended with financial language.

Coherence Scores measure topic interpretability and semantic consistency. Higher scores indicate more coherent topics.


#### Environment: 0.4339


Moderate coherence indicates distinct but overlapping themes linking financial and environmental reporting.


#### Social: 0.4402


Interpretation: Balanced topic spread reflecting employee-centric and governance-linked social reporting practices.


#### Governance: 0.4918


Highest coherence among the three pillars, indicating well-defined governance discussions centered on compliance and financial control.

In [54]:
# Based on the topic modeling results, defining dominant topics for each ESG dimension
# Used AI assistance to interpret and label topics according to the top keywords and thematic coherence.
dominant_topics = {
    "E": {
        0: "Climate & Emissions Regulation and Compliance",
        1: "Energy and Resource Management",
        2: "Environmental Sustainability and Strategic Reporting"
    },
    "S": {
        0: "Workforce and Labor Management",
        1: "Diversity, Inclusion, and Executive Governance",
        2: "Health, Safety, and Operational Performance"
    },
    "G": {
        0: "Board Leadership and Internal Control",
        1: "Ethics, Compliance, and Executive Oversight",
        2: "Financial Risk, Security, and Reporting Integrity"
    }
}


In [55]:
def map_doc_level_topics(df_original, all_results, topic_name_map=None):
    """
    Merges document-level topic assignments from multiple ESG dimensions into the original dataframe by mapping topic IDs to dominant themes.
    -----------
    Parameters:
    -----------
    df_original : pd.DataFrame
        Original dataframe containing filings.
    all_results : dict
        Dictionary containing results from topic modeling for each ESG dimension.
    topic_name_map : dict, optional
        Mapping of topic IDs to human-readable names for each dimension.
        
    Returns:
    --------
    pd.DataFrame
        DataFrame with merged topic assignments and mapped topic names.
    """
    df = df_original.copy().reset_index(drop=True)
    # For each ESG dimension, merging the document-topic assignments
    for dim, res in (all_results or {}).items():
        doc_df = res.get('doc_topics')
        if doc_df is None:
            continue
        # Making a copy of the document-level dataframe
        doc_df = doc_df.copy()
        # Check if dominant_topic column exists        
        if 'dominant_topic' not in doc_df.columns:
            continue
        # Keep only necessary columns
        keep = ['ticker', 'filing_date', 'dominant_topic']
        if 'dominant_topic_name' in doc_df.columns:
            keep.append('dominant_topic_name')
        doc_df = doc_df[keep].drop_duplicates()
        # Renaming columns to reflect ESG dimension
        doc_df = doc_df.rename(columns={
            'dominant_topic': f'{dim}_topic_id',
            'dominant_topic_name': f'{dim}_topic_name'
        })

        # Merging with the original dataframe
        df = df.merge(doc_df, how='left', on=['ticker', 'filing_date'])

        # Ensuring topic names are mapped correctly
        topic_col = f'{dim}_topic_id'
        name_col = f'{dim}_topic_name'
        if topic_col in df.columns:
            if name_col not in df.columns:
                df[name_col] = None
            df[name_col] = df[topic_col].apply(
                lambda t: topic_name_map.get(dim, {}).get(int(t), f'{dim}_T{int(t)}')
                if pd.notna(t) else None
            )
    return df


In [56]:
df_mapped = map_doc_level_topics(df, all_results, topic_name_map=dominant_topics)

In [57]:
df_mapped[['ticker', 'filing_date', 'E_topic_id', 'E_topic_name',
           'S_topic_id', 'S_topic_name', 'G_topic_id', 'G_topic_name']]

,ticker,filing_date,E_topic_id,E_topic_name,S_topic_id,S_topic_name,G_topic_id,G_topic_name
0,AAPL,2015-10-28,0.0,Climate & Emissions Regulation and Compliance,2.0,"Health, Safety, and Operational Performance",2,"Financial Risk, Security, and Reporting Integrity"
1,AAPL,2016-10-26,0.0,Climate & Emissions Regulation and Compliance,0.0,Workforce and Labor Management,2,"Financial Risk, Security, and Reporting Integrity"
2,AAPL,2017-11-03,0.0,Climate & Emissions Regulation and Compliance,1.0,"Diversity, Inclusion, and Executive Governance",2,"Financial Risk, Security, and Reporting Integrity"
3,AAPL,2018-11-05,0.0,Climate & Emissions Regulation and Compliance,0.0,Workforce and Labor Management,0,Board Leadership and Internal Control
4,AAPL,2019-10-31,0.0,Climate & Emissions Regulation and Compliance,0.0,Workforce and Labor Management,2,"Financial Risk, Security, and Reporting Integrity"
...,...,...,...,...,...,...,...,...
492,INTU,2020-08-31,0.0,Climate & Emissions Regulation and Compliance,1.0,"Diversity, Inclusion, and Executive Governance",0,Board Leadership and Internal Control
493,INTU,2021-09-08,0.0,Climate & Emissions Regulation and Compliance,1.0,"Diversity, Inclusion, and Executive Governance",0,Board Leadership and Internal Control
494,INTU,2022-09-02,0.0,Climate & Emissions Regulation and Compliance,1.0,"Diversity, Inclusion, and Executive Governance",0,Board Leadership and Internal Control
495,INTU,2023-09-01,0.0,Climate & Emissions Regulation and Compliance,1.0,"Diversity, Inclusion, and Executive Governance",0,Board Leadership and Internal Control


In [58]:
df_mapped.sample(10)

,ticker,filing_date,text,cleaned_report_text,cleaned_sentences,ESG_sentences,ESG_dimension,E_topic_id,E_topic_name,S_topic_id,S_topic_name,G_topic_id,G_topic_name
480,LOW,2018-04-02,...,united states securities and exchange commissi...,[united states securities and exchange commiss...,[• item 9 - changes in and disagreements with ...,"[(E, emissions), (E, water), (E, waste), (S, l...",2.0,Environmental Sustainability and Strategic Rep...,1.0,"Diversity, Inclusion, and Executive Governance",0,Board Leadership and Internal Control
217,COST,2016-10-21,...,united states securities and exchange commissi...,[united states securities and exchange commiss...,[in discussions of our consolidated operating ...,"[(S, labor), (G, board), (G, transparency)]",NaN,None,0.0,Workforce and Labor Management,2,"Financial Risk, Security, and Reporting Integrity"
273,ACN,2021-10-15,Table of Contents\n ...,table of contents united states securities and...,[table of contents united states securities an...,"[• item 9c., we also have a number of contract...","[(E, emissions), (E, energy), (E, climate), (E...",0.0,Climate & Emissions Regulation and Compliance,1.0,"Diversity, Inclusion, and Executive Governance",2,"Financial Risk, Security, and Reporting Integrity"
281,ABT,2019-02-22,UNITED STATES\nSECURITIES AND EXCHANGE COMMISS...,united states securities and exchange commissi...,[united states securities and exchange commiss...,[** *form of agreement regarding change in con...,"[(S, labor), (S, diversity), (S, customer), (S...",0.0,Climate & Emissions Regulation and Compliance,0.0,Workforce and Labor Management,0,Board Leadership and Internal Control
436,AMD,2024-01-31,...,united states securities and exchange commissi...,[united states securities and exchange commiss...,[compensation expense is recognized over the v...,"[(E, emissions), (E, energy), (E, climate), (E...",0.0,Climate & Emissions Regulation and Compliance,1.0,"Diversity, Inclusion, and Executive Governance",0,Board Leadership and Internal Control
265,TMO,2023-02-23,...,united states securities and exchange commissi...,[united states securities and exchange commiss...,[the company’s financing activities also inclu...,"[(E, climate), (E, waste), (E, pollution), (S,...",0.0,Climate & Emissions Regulation and Compliance,0.0,Workforce and Labor Management,2,"Financial Risk, Security, and Reporting Integrity"
191,KO,2018-02-23,...,united states securities and exchange commissi...,[united states securities and exchange commiss...,[for information regarding the estimated impac...,"[(E, emissions), (E, energy), (E, climate), (E...",1.0,Energy and Resource Management,2.0,"Health, Safety, and Operational Performance",2,"Financial Risk, Security, and Reporting Integrity"
368,NEE,2016-02-22,...,table of contents united states securities and...,[table of contents united states securities an...,[stock-based compensation - nee accounts for s...,"[(E, emissions), (E, energy), (E, climate), (E...",1.0,Energy and Resource Management,1.0,"Diversity, Inclusion, and Executive Governance",2,"Financial Risk, Security, and Reporting Integrity"
117,V,2016-11-15,Table of Contents\n ...,table of contents united states securities and...,[table of contents united states securities an...,[the company’s contributions are funded on a c...,"[(E, biodiversity), (S, labor), (S, diversity)...",1.0,Energy and Resource Management,1.0,"Diversity, Inclusion, and Executive Governance",0,Board Leadership and Internal Control
286,ABT,2024-02-16,Table of Contents\n ...,table of contents united states securities and...,[table of contents united states securities an...,[the 2024 proxy statement will be filed on or ...,"[(E, climate), (S, labor), (S, diversity), (S,...",0.0,Climate & Emissions Regulation and Compliance,0.0,Workforce and Labor Management,2,"Financial Risk, Security, and Reporting Integrity"


In [59]:
df_mapped['E_topic_name'].value_counts()

E_topic_name
Climate & Emissions Regulation and Compliance           211
Environmental Sustainability and Strategic Reporting    171
Energy and Resource Management                           92
Name: count, dtype: int64

In [60]:
df_mapped['S_topic_name'].value_counts()

S_topic_name
Diversity, Inclusion, and Executive Governance    234
Workforce and Labor Management                    156
Health, Safety, and Operational Performance       106
Name: count, dtype: int64

In [61]:
df_mapped['G_topic_name'].value_counts()

G_topic_name
Board Leadership and Internal Control                259
Financial Risk, Security, and Reporting Integrity    187
Ethics, Compliance, and Executive Oversight           51
Name: count, dtype: int64

In [62]:
df_mapped['ESG_topic_combination']=df_mapped['E_topic_name']+" | "+df_mapped['S_topic_name']+" | "+df_mapped['G_topic_name']

In [63]:
ticker_data = {
    'AAPL': ['Apple Inc.', 'Information Tech'],
    'MSFT': ['Microsoft Corp', 'Information Tech'],
    'GOOGL': ['Alphabet Inc. Class A', 'Communication Svcs'],
    'AMZN': ['Amazon.com Inc.', 'Consumer Discret.'],
    'NVDA': ['NVIDIA Corporation', 'Information Tech'],
    'META': ['Meta Platforms, Inc.', 'Communication Svcs'],
    'TSLA': ['Tesla, Inc.', 'Consumer Discret.'],
    'UNH': ['UnitedHealth Group', 'Health Care'],
    'XOM': ['Exxon Mobil Corp', 'Energy'],
    'JNJ': ['Johnson & Johnson', 'Health Care'],
    'JPM': ['JPMorgan Chase & Co.', 'Financials'],
    'V': ['Visa Inc.', 'Information Tech'],
    'PG': ['Procter & Gamble Co.', 'Consumer Staples'],
    'MA': ['Mastercard Inc.', 'Information Tech'],
    'HD': ['Home Depot, Inc.', 'Consumer Discret.'],
    'CVX': ['Chevron Corporation', 'Energy'],
    'MRK': ['Merck & Co., Inc.', 'Health Care'],
    'ABBV': ['AbbVie Inc.', 'Health Care'],
    'KO': ['Coca-Cola Company', 'Consumer Staples'],
    'AVGO': ['Broadcom Inc.', 'Information Tech'],
    'PEP': ['PepsiCo, Inc.', 'Consumer Staples'],
    'COST': ['Costco Wholesale Corp', 'Consumer Staples'],
    'WMT': ['Walmart Inc.', 'Consumer Staples'],
    'MCD': ['McDonald\'s Corp', 'Consumer Discret.'],
    'CSCO': ['Cisco Systems, Inc.', 'Information Tech'],
    'TMO': ['Thermo Fisher Scientific', 'Health Care'],
    'ACN': ['Accenture Plc', 'Information Tech'],
    'ABT': ['Abbott Laboratories', 'Health Care'],
    'DHR': ['Danaher Corporation', 'Health Care'],
    'NKE': ['NIKE, Inc.', 'Consumer Discret.'],
    'CRM': ['Salesforce, Inc.', 'Information Tech'],
    'VZ': ['Verizon Communications', 'Communication Svcs'],
    'ADBE': ['Adobe Inc.', 'Information Tech'],
    'CMCSA': ['Comcast Corporation', 'Communication Svcs'],
    'WFC': ['Wells Fargo & Co.', 'Financials'],
    'TXN': ['Texas Instruments Inc.', 'Information Tech'],
    'NEE': ['NextEra Energy, Inc.', 'Utilities'],
    'LIN': ['Linde plc', 'Materials'],
    'PM': ['Philip Morris International', 'Consumer Staples'],
    'NFLX': ['Netflix, Inc.', 'Communication Svcs'],
    'UPS': ['United Parcel Service', 'Industrials'],
    'RTX': ['Raytheon Technologies', 'Industrials'],
    'AMD': ['Advanced Micro Devices', 'Information Tech'],
    'QCOM': ['Qualcomm Incorporated', 'Information Tech'],
    'ORCL': ['Oracle Corporation', 'Information Tech'],
    'HON': ['Honeywell International', 'Industrials'],
    'SPGI': ['S&P Global Inc.', 'Financials'],
    'LOW': ['Lowe\'s Companies, Inc.', 'Consumer Discret.'],
    'INTU': ['Intuit Inc.', 'Information Tech']
}
# Creating a DataFrame from the ticker data dictionary
df_tickers = pd.DataFrame.from_dict(ticker_data, orient='index', columns=['Company Name', 'Sector'])


In [64]:
# Merging the ticker information with the main DataFrame
df_mapped=df_mapped.merge(df_tickers, how='left', left_on='ticker', right_index=True)

In [65]:
df_mapped.to_csv("Final_mapped_topics.csv", index=False)

In [66]:
# Hedge words are words that indicate uncertainty or caution in statements.
HEDGE_WORDS = {
    # Modal verbs
    "may", "might", "could", "would", "should", "can", "cannot",
    # Probability/adverbs
    "possibly", "likely", "probably", "uncertain", "uncertainty", 
    "apparently", "seemingly", "apparently", "potentially", 
    "potential", "approximately", "roughly", "estimated", "estimates",
    # Verbs indicating caution or inference
    "appear", "appears", "seem", "seems", "suggest", "indicate", 
    "believe", "expects", "expect", "anticipate", "intend", "assume", 
    "assumes", "assumed", "forecast", "projected", "predict", "predicted",
    # Phrases often used as hedges
    "it is possible that", "there is a risk that", "it appears that", 
    "it seems that", "it may be", "it might be", "it could be",
    # Qualifiers/adjectives
    "probable", "possible", "uncertain", "tentative", "preliminary", 
    "speculative", "approximate", "indicative", "conditional"
}


In [67]:
# Vague words are words that lack specificity and clarity.
VAGUE_WORDS = {
    # Quantifiers / general amounts
    "various", "several", "many", "some", "certain", "numerous", 
    "a number of", "a few", "a variety of", "several types", "several kinds", 
    "diverse", "multiple", "countless", "frequent", "occasional",

    # General adjectives / weak descriptors
    "significant", "substantial", "material", "considerable", "adequate", 
    "appropriate", "relevant", "important", "key", "major", "minor", 
    "limited", "moderate", "reasonable", "sufficient", "extensive", 
    "broad", "wide-ranging", "notable", "somewhat", "relatively", 
    "typical", "common", "general", "varied", "variously",

    # Ambiguous nouns
    "issue", "factor", "aspect", "element", "component", "part", 
    "item", "feature", "process", "activity", "area", "field",

    # Phrases indicating lack of specificity
    "in some cases", "in certain cases", "to some extent", 
    "to a certain degree", "in general", "for the most part", 
    "as appropriate", "as relevant", "as necessary", "as applicable",

    # Hedging / uncertainty (overlaps with hedge words)
    "may", "might", "could", "should", "possibly", "likely", 
    "potential", "probable", "uncertain", "appears", "seems", 
    "suggest", "indicate", "believe", "expect", "anticipate", "intend"

}


In [68]:
# Additional regex patterns to identify specific numeric and financial terms in ESG contexts
UNIT_PATTERN = re.compile(
    r'\b\d+(?:,\d{3})*(?:\.\d+)?\s*'            # Number with optional commas and decimals
    r'(?:ton|tons|tco2e|tonnes|kg|g|lb|lbs|mwh|gwh|kwh|mw|gw|%'       # Mass / Energy / Power / %
    r'|percent|pp|bps|\$|usd|million|billion|thousand|km|m|cm|inch|ft)'  # Currency / scale / distance / length
    r'\b',
    flags=re.I
)
NUMBER_PATTERN = re.compile(
    r'\b\d{1,3}(?:,\d{3})*(?:\.\d+)?\b'   # Matches 1, 10, 1,000, 10.5, 1,234.56
)
YEAR_PATTERN = re.compile(
    r'\b(?:19|20)\d{2}\b'   # Matches years 1900–2099
)
FINANCIAL_PATTERN = re.compile(
    r'\b(?:revenue|sales|profit|net income|ebitda|eps|assets|liabilities|equity|capex|opex|dividend)\b',
    flags=re.I
)
ESG_METRIC_PATTERN = re.compile(
    r'\b(?:carbon emissions|ghg emissions|co2 emissions|energy consumption|water usage|waste generated|scope 1|scope 2|scope 3|emissions intensity)\b',
    flags=re.I
)

In [69]:
def calculate_concreteness_score(text):
    """
    Calculates a concreteness score for the given text based on the presence of specific linguistic features.
    -----------
    Parameters:
    text (str): Input text to evaluate.
    -----------
    Returns:
    float: Concreteness score between 0.0 (very vague) and 1
    -----------
    Example Usage:
    score = calculate_concreteness_score(text)
    """
    # Converting text to lowercase and tokenizing into words
    text_str = str(text)
    text_lower = text_str.lower()
    words = re.findall(r'\b\w+\b', text_lower)
    word_count = len(words)
    # Avoid division by zero
    if word_count == 0:
        return 0.90
    # Concreteness features indicating specificity
    numbers = len(NUMBER_PATTERN.findall(text_str))
    units = len(UNIT_PATTERN.findall(text_str))
    years = len(YEAR_PATTERN.findall(text_str))
    # Vagueness features indicating lack of specificity
    hedges = sum(1 for w in words if w in HEDGE_WORDS)
    vague = sum(1 for w in words if w in VAGUE_WORDS)
    # Calculate densities of features
    number_density = numbers / word_count
    unit_density = units / word_count
    year_density = years / word_count
    hedge_density = hedges / word_count
    vague_density = vague / word_count
    # Concreteness score: higher numbers, units, and years increase concreteness
    concreteness = (number_density * 8 + unit_density * 12 + year_density * 7)
    # Vagueness score: hedge words and vague terms reduce concreteness
    vagueness = (hedge_density * 14 + vague_density * 10)
    # Overall score: base 0.5, adjusted positively by concreteness and negatively by vagueness
    score = 0.5 + (concreteness * 0.4) - (vagueness * 0.4)
    return float(np.clip(score, 0.0, 1.0))
    

In [70]:
# Calculating concreteness and vague scores for each ESG sentence in the dataframe
df['concreteness_score'] = df['ESG_sentences'].apply(calculate_concreteness_score)
df['vague_score'] = 1 - df['concreteness_score']

In [71]:
# Calculating concreteness and vague scores for the entire cleaned report text
df['concreteness_score_report'] = df['cleaned_report_text'].apply(calculate_concreteness_score)
df['vague_score_report'] = 1 - df['concreteness_score_report']

In [72]:
# Displaying scores for Apple Inc. (AAPL) as an example
df[df['ticker']=="AAPL"][['ticker','filing_date','concreteness_score','vague_score','concreteness_score_report','vague_score_report']]

,ticker,filing_date,concreteness_score,vague_score,concreteness_score_report,vague_score_report
0,AAPL,2015-10-28,0.737682,0.262318,0.622073,0.377927
1,AAPL,2016-10-26,0.715013,0.284987,0.617665,0.382335
2,AAPL,2017-11-03,0.736297,0.263703,0.628905,0.371095
3,AAPL,2018-11-05,0.725134,0.274866,0.617788,0.382212
4,AAPL,2019-10-31,0.721576,0.278424,0.635465,0.364535
5,AAPL,2020-10-30,0.697329,0.302671,0.631527,0.368473
6,AAPL,2021-10-29,0.645982,0.354018,0.617848,0.382152
7,AAPL,2022-10-28,0.646617,0.353383,0.617018,0.382982
8,AAPL,2023-11-03,0.651483,0.348517,0.637683,0.362317
9,AAPL,2024-11-01,0.613600,0.386400,0.627575,0.372425


In [73]:
def calculate_pillar_concreteness(text, esg_keywords):
    """
    Calculates concreteness scores for each ESG pillar (E, S, G) based on the provided text.
    -----------
    Parameters:
    text (str): Text to analyze for concreteness.
    esg_keywords (dict): Dictionary containing ESG-related keywords for each pillar.
    -----------
    Returns:
    dict: Dictionary with concreteness scores for each pillar.
    -----------
    Example Usage:
    pillar_scores = calculate_pillar_concreteness(text, esg_keywords)
    """
    # Converting to lowercase and split into sentences
    text_str = str(text)
    doc = text_str.split('.')  
    pillar_scores = defaultdict(list)
    # Analyzing each sentence
    for sent in doc:
        sent_lower = sent.lower()
        words = re.findall(r'\b\w+\b', sent_lower)
        wc = len(words)
        if wc == 0:
            continue
        # Determining pillar assignment by keywords
        assigned = None
        for p, kws in esg_keywords.items():
            if any(k.lower() in sent_lower for k in kws):
                assigned = p
                break
        if not assigned:
            continue
        # Concreteness features
        numbers = len(NUMBER_PATTERN.findall(sent))
        units = len(UNIT_PATTERN.findall(sent))
        years = len(YEAR_PATTERN.findall(sent))
        # Vagueness features
        hedges = sum(1 for w in words if w in HEDGE_WORDS)
        vague = sum(1 for w in words if w in VAGUE_WORDS)
        # Densities
        number_density = numbers / wc
        unit_density = units / wc
        year_density = years / wc
        hedge_density = hedges / wc
        vague_density = vague / wc
        concreteness = (
            0.5
            + (number_density * 0.4)
            + (unit_density * 0.4)
            + (year_density * 0.3)
            - (hedge_density * 0.4)
            - (vague_density * 0.6)
        )
        concreteness = float(np.clip(concreteness, 0.0, 1.0))
        pillar_scores[assigned].append(concreteness)
    # Averaging scores for each pillar
    result = {}
    for p in ['E','S','G']:
        if pillar_scores[p]:
            result[f'{p}_concrete'] = round(np.mean(pillar_scores[p]), 3)
        else:
            result[f'{p}_concrete'] = 0.0
    return result


In [74]:
# Calculating pillar-specific concreteness scores for each document as it determines how specifically each ESG pillar is addressed in the ESG framework.
df[['E_concrete','S_concrete','G_concrete']] = df['ESG_sentences'].apply(
    lambda x: pd.Series(calculate_pillar_concreteness(x, esg_keywords))
)


In [75]:
# Displaying pillar-specific concreteness scores for Apple Inc. (AAPL) and Microsoft Corp. (MSFT) as examples
df[['ticker','filing_date','concreteness_score','vague_score','E_concrete','S_concrete','G_concrete']][df['ticker'].isin(["AAPL", "MSFT"])]

,ticker,filing_date,concreteness_score,vague_score,E_concrete,S_concrete,G_concrete
0,AAPL,2015-10-28,0.737682,0.262318,0.493,0.476,0.515
1,AAPL,2016-10-26,0.715013,0.284987,0.500,0.467,0.518
2,AAPL,2017-11-03,0.736297,0.263703,0.500,0.477,0.516
3,AAPL,2018-11-05,0.725134,0.274866,0.416,0.478,0.511
4,AAPL,2019-10-31,0.721576,0.278424,0.416,0.488,0.518
5,AAPL,2020-10-30,0.697329,0.302671,0.416,0.488,0.514
6,AAPL,2021-10-29,0.645982,0.354018,0.484,0.492,0.497
7,AAPL,2022-10-28,0.646617,0.353383,0.484,0.482,0.502
8,AAPL,2023-11-03,0.651483,0.348517,0.487,0.487,0.503
9,AAPL,2024-11-01,0.613600,0.386400,0.486,0.488,0.497


In [76]:
# Counting tokens for each ESG dimension based on the ESG_sentences and ESG_dimension columns
# Token counts provide insight into the amount of content dedicated to each ESG pillar.
for dim in ['E', 'S', 'G']:
    df[f'{dim}_tokens'] = df.apply(
        lambda row: sum(len(row['cleaned_sentences'][i].split())
                        for i, (d, _) in enumerate(row['ESG_dimension']) if d == dim) 
        if isinstance(row['cleaned_sentences'], list) and isinstance(row['ESG_dimension'], list) else 0,
        axis=1
    )

In [77]:
# Displaying token counts for Apple Inc. (AAPL) and Microsoft Corp. (MSFT) as examples
df[['ticker','filing_date','E_tokens','S_tokens','G_tokens']][df['ticker'].isin(["AAPL", "MSFT"])]

,ticker,filing_date,E_tokens,S_tokens,G_tokens
0,AAPL,2015-10-28,86,294,286
1,AAPL,2016-10-26,86,288,246
2,AAPL,2017-11-03,83,311,282
3,AAPL,2018-11-05,83,386,205
4,AAPL,2019-10-31,81,265,251
5,AAPL,2020-10-30,81,283,225
6,AAPL,2021-10-29,81,362,240
7,AAPL,2022-10-28,84,382,259
8,AAPL,2023-11-03,84,372,271
9,AAPL,2024-11-01,84,362,271


In [78]:
# Counting total tokens in cleaned sentences for each document
df['cleaned_token_count'] = df['cleaned_sentences'].apply(lambda x: sum(len(s.split()) for s in x) if isinstance(x, list) else 0)

In [79]:
#Calculating the overall ESG percentage based on word counts in each dimension
df['total_esg_tokens'] = df['E_tokens'] + df['S_tokens'] + df['G_tokens']
df['Overall_ESG_content'] = np.where(
    df['cleaned_token_count'] > 0,
    (df['total_esg_tokens'] / df['cleaned_token_count'])*100,
    0
).round(5)

In [80]:
# Displaying overall ESG content percentages for Apple Inc. (AAPL) and Microsoft Corp. (MSFT) as examples
df[['ticker', 'filing_date','Overall_ESG_content']][df['ticker'].isin(["AAPL", "MSFT"])]

,ticker,filing_date,Overall_ESG_content
0,AAPL,2015-10-28,1.47332
1,AAPL,2016-10-26,1.38347
2,AAPL,2017-11-03,1.47595
3,AAPL,2018-11-05,1.56770
4,AAPL,2019-10-31,1.65122
5,AAPL,2020-10-30,1.61379
6,AAPL,2021-10-29,2.04332
7,AAPL,2022-10-28,2.25002
8,AAPL,2023-11-03,2.42261
9,AAPL,2024-11-01,2.34920


In [81]:
# Calculating percentage distribution of content across E, S, and G dimensions
esg_merge=esg_percentage_dist[['ticker','filing_date','E_percentage','S_percentage','G_percentage']]
df_final=df.merge(esg_merge, how='left', on=['ticker','filing_date'])

In [82]:
# Calculating percentage of Environmental (E) content in the report
df['E_percentage_in_report'] = np.where(
    df['cleaned_token_count'] > 0,
    (df['E_tokens'] / df['cleaned_token_count']) * 100,
    0
).round(5)


In [83]:
# Calculating percentage of Social (S) content in the report
df['S_percentage_in_report'] = np.where(
    df['cleaned_token_count'] > 0,
    (df['S_tokens'] / df['cleaned_token_count']) * 100,
    0
).round(5)


In [84]:
# Calculating percentage of Governance (G) content in the report
df['G_percentage_in_report'] = np.where(
    df['cleaned_token_count'] > 0,
    (df['G_tokens'] / df['cleaned_token_count']) * 100,
    0
).round(5)


In [85]:
# Calculating pillar-specific concreteness scores for the entire cleaned report text
df[['E_concrete_report','S_concrete_report','G_concrete_report']] = df['cleaned_report_text'].apply(
    lambda x: pd.Series(calculate_pillar_concreteness(x, esg_keywords))
)


In [86]:
# Displaying final ESG metrics for Apple Inc. (AAPL) and Microsoft Corp. (MSFT) as examples
df[['ticker','filing_date','E_tokens','S_tokens','G_tokens','E_concrete_report','S_concrete_report','G_concrete_report','Overall_ESG_content','concreteness_score_report','concreteness_score']][df['ticker'].isin(["AAPL", "MSFT"])]

,ticker,filing_date,E_tokens,S_tokens,G_tokens,E_concrete_report,S_concrete_report,G_concrete_report,Overall_ESG_content,concreteness_score_report,concreteness_score
0,AAPL,2015-10-28,86,294,286,0.496,0.488,0.496,1.47332,0.622073,0.737682
1,AAPL,2016-10-26,86,288,246,0.510,0.489,0.495,1.38347,0.617665,0.715013
2,AAPL,2017-11-03,83,311,282,0.488,0.493,0.494,1.47595,0.628905,0.736297
3,AAPL,2018-11-05,83,386,205,0.447,0.486,0.492,1.56770,0.617788,0.725134
4,AAPL,2019-10-31,81,265,251,0.447,0.489,0.494,1.65122,0.635465,0.721576
5,AAPL,2020-10-30,81,283,225,0.447,0.488,0.492,1.61379,0.631527,0.697329
6,AAPL,2021-10-29,81,362,240,0.487,0.487,0.490,2.04332,0.617848,0.645982
7,AAPL,2022-10-28,84,382,259,0.486,0.486,0.492,2.25002,0.617018,0.646617
8,AAPL,2023-11-03,84,372,271,0.487,0.487,0.491,2.42261,0.637683,0.651483
9,AAPL,2024-11-01,84,362,271,0.486,0.485,0.491,2.34920,0.627575,0.613600


In [87]:
# Calculating GRI (Greenwashing Risk Index) score as a measure of ESG reporting quality.
df['GRI']=(df['Overall_ESG_content']/100)/(df['concreteness_score_report']+0.01)

In [88]:
# Displaying GRI and related ESG metrics for Apple Inc. (AAPL) as an example
df[df['ticker']=="AAPL"][['ticker','filing_date','GRI','Overall_ESG_content','concreteness_score_report']]

,ticker,filing_date,GRI,Overall_ESG_content,concreteness_score_report
0,AAPL,2015-10-28,0.023309,1.47332,0.622073
1,AAPL,2016-10-26,0.022042,1.38347,0.617665
2,AAPL,2017-11-03,0.023101,1.47595,0.628905
3,AAPL,2018-11-05,0.024972,1.56770,0.617788
4,AAPL,2019-10-31,0.025582,1.65122,0.635465
5,AAPL,2020-10-30,0.025155,1.61379,0.631527
6,AAPL,2021-10-29,0.032545,2.04332,0.617848
7,AAPL,2022-10-28,0.035884,2.25002,0.617018
8,AAPL,2023-11-03,0.037404,2.42261,0.637683
9,AAPL,2024-11-01,0.036846,2.34920,0.627575


In [89]:
# Displaying GRI and related ESG metrics for Tesla Inc. (TSLA) as an example
df[df['ticker']=="TSLA"][['ticker','filing_date','GRI','Overall_ESG_content','concreteness_score_report']]

,ticker,filing_date,GRI,Overall_ESG_content,concreteness_score_report
68,TSLA,2015-02-26,0.021105,1.04372,0.484536
69,TSLA,2016-02-24,0.023036,1.28368,0.547240
70,TSLA,2017-03-01,0.012038,0.77989,0.637883
71,TSLA,2018-02-23,0.011826,0.76446,0.636438
72,TSLA,2019-02-19,0.011821,0.74773,0.622567
73,TSLA,2020-02-13,0.012860,0.75864,0.579901
74,TSLA,2020-04-28,0.018366,1.50849,0.811370
75,TSLA,2021-02-08,0.014941,0.86564,0.569390
76,TSLA,2021-04-30,0.020401,1.60769,0.778053
77,TSLA,2022-02-07,0.019042,1.13069,0.583797


In [90]:
df.to_csv("ESG_Topic_Modeling_Concreteness_Scores.csv", index=False)

In [91]:
df.columns

Index(['ticker', 'filing_date', 'text', 'cleaned_report_text',
       'cleaned_sentences', 'ESG_sentences', 'ESG_dimension',
       'concreteness_score', 'vague_score', 'concreteness_score_report',
       'vague_score_report', 'E_concrete', 'S_concrete', 'G_concrete',
       'E_tokens', 'S_tokens', 'G_tokens', 'cleaned_token_count',
       'total_esg_tokens', 'Overall_ESG_content', 'E_percentage_in_report',
       'S_percentage_in_report', 'G_percentage_in_report', 'E_concrete_report',
       'S_concrete_report', 'G_concrete_report', 'GRI'],
      dtype='object')